Plano de hoje
-------------

1. Ambiente de programação
2. Usando o computador para calcular    
3. Usando o computador para desenhar
4. Usando o computador para integrar: quadraturas
5. Usando o computador para aproximar: interpolação
6. Álgebra linear computacional
    1. Resolvendo sistemas lineares

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Sistemas lineares

Um sistema linear é uma equação da forma $Ax = b$ onde $A$ e $b$ são dados, e $x$ é uma incógnita.
Normalmente, só usamos este nome quando $A$ é uma matriz e $b$ um vetor, e portanto $x$ será um vetor também,
mas em geral poderíamos tanto usar o caso em que $b$ é "maior" (por exemplo, uma matriz também)
quando o caso em que $A$ é menor (um vetor, ou até mesmo um escalar).
Entretanto, os casos em que $A$ é "pequena" são "fáceis",
e o caso em que $b$ é grande não apresenta mais dificuldade do que o caso normal.

Já encontramos alguns exemplos em que aparecem equações lineares,
por exemplo na interpolação de splines ou em mínimos quadrados.
(Lembre que o caso da interpolação de Lagrange ou FFT a solução do sistema linear é explícita,
devido a sua forma especial, o que já não é o caso das outras duas acima citadas.)

## Observação

Como vários outros problemas matemáticos deste curso,
também as questões relativas a sistemas lineares já foram _muito_ estudadas.
Assim, não é de se espantar que, da mesma forma que já encontramos a função
`scipy.interpolate.interp1d` para calcular splines,
também já há diversas funções (desta vez, no `numpy` mesmo, em geral)
que tratam de sistemas lineares e problemas matriciais em geral.

Vamos (re-)implementar algumas delas (e muitas vezes de forma menos eficiente ou menos correta)
para compreender o mecanismo e os algoritmos.
Mas, provavelmente, tanto por questões de velocidade como de praticidade,
será bastante útil conhecer as funções do `numpy.linalg`.

## Eliminação

O primeiro algoritmo de solução de sistemas lineares é conhecido por "eliminação de Gauss",
apesar de ter sido descoberto pela primeira vez na China por volta de 150 a.C.
e aparecer sob uma forma relativamente moderna com Newton (uns 150 anos antes de Gauss).

A idéia é bastante simples:
seja um sistema com $n$ equações e $m$ incógnitas.
Usa-se uma das equações (a primeira, em geral) para escrever uma das variáveis
(também em geral a primeira) em função das outras.
Em seguida, substitui-se esta variável em todas as outras equações,
obtendo um sistema com $n-1$ equações e $m-1$ incógnitas,
após reagrupar os coeficientes de cada termo.

Exemplo:
$$ \begin{align*}
   x + 2y + 3z & = 10 \\
  4x + 5y + 6z & = 15
\end{align*} $$
Obtemos $x = 10 - 2y - 3z$ que na segunda equação dá
$$ \begin{align*}
  4(10 - 2y - 3z) + 5y + 6z & = 15 \\
  40 - 8y - 12z + 5y + 6z & = 15 \\
  -3y - 6z & = - 25
\end{align*} $$

## Industrializando a eliminação: notação matricial

Um dos méritos de Gauss foi ter inventado a notação matricial
(inicialmente, para resolver problemas de aritmética!)
e com isso poupar muito espaço e tempo ao se resolver sistemas.
Com esta nova roupagem, foi possível refinar o método acima para torná-lo
praticamente automático.

Observe que _eliminar a primeira variável da segunda equação, usando a primeira_
pode ser obtido ao multiplicar a primeira equação por $-4$ e somar à segunda.
Assim, podemos trabalhar unicamente com os coeficientes
(aliás, se você pensar bem, o **nome** que damos às incógnitas é totalmente irrelevante)
num "dispositivo prático":

$$ \begin{bmatrix}1 & 2 & 3 & \vert & 10 \\ 4 & 5 & 6 & \vert & 15 \end{bmatrix} $$
$$ \begin{bmatrix}1 & 2 & 3 & \vert & 10 \\
  4 + (-4)\times 1 & 5 + (-4)\times 2 & 6 + (-4) \times 3 & \vert & 15 + (-4)\times 10
  \end{bmatrix} $$

### Exercício

Implemente a eliminação para matrizes.

In [2]:
def elim(A,b):
    """ Elimina as equações do sistema  Ax = b.
        A matriz  A  será modificada, ficando sob forma triangular superior, e b será o vetor correspondente ao novo sistema. """
    # Os coeficientes de A estão em A[i][j]
    #for i in #:
    #    for ii in #:
    #        for j in #:
    #
    # Como A e b serão modificados, não é preciso retorná-los

Para testar, use o sistema que já resolvemos:

In [5]:
A = array([[1, 2, 3], [4, 5, 6]])
b = array([10, 25])
elim(A,b)
print(A)
print(b)

[[ 1  2  3]
 [ 0 -3 -6]]
[ 10 -15]


In [6]:
A = array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = array([10, 25, 12])
elim(A,b)
print(A)
print(b)

[[ 1  2  3]
 [ 0 -3 -6]
 [ 0  0  0]]
[ 10 -15 -28]


E quando você estiver confiante, elimine em sistemas de maior dimensão!

In [9]:
A = rand(8, 10)
b = rand(12)

In [10]:
elim(A,b)
np.set_printoptions(precision=4,linewidth=100)#, suppress=True) # Para ficar com cara de matriz no print
print(A)
print(b)

[[ 0.1832  0.7933  0.0568  0.8782  0.5663  0.8423  0.2595  0.4746  0.1712  0.1462]
 [ 0.     -1.7939  0.7184 -1.9777 -1.0799 -2.1284  0.0775 -1.3284 -0.4221 -0.0418]
 [ 0.      0.     -0.109   0.4251  0.8525  0.1706  0.1721  0.1686  0.9278 -0.0392]
 [ 0.      0.      0.      1.9557  4.7136  1.0244  1.293   0.5229  5.0329  0.3925]
 [ 0.      0.      0.      0.      1.2533  0.6191  0.4526  0.1231  1.6812  0.6898]
 [ 0.      0.      0.      0.      0.      0.5172  0.1845  0.6758  0.9668  0.4971]
 [ 0.      0.      0.      0.      0.      0.      0.4858  0.5993  0.4787  0.5099]
 [ 0.      0.      0.      0.      0.      0.      0.      0.7929  0.8462  0.1391]]
[ 0.9829 -2.3143  0.9138  4.9289  1.2226 -0.24   -0.1387 -0.5481  0.4224  0.7717  0.9484  0.1973]


## Substituição

Ao eliminar, obtemos um sistema cada vez menor, até que ele tenha $n-m$ equações e $0$ incógnitas,
ou o contrário: $0$ equações e $m-n$ incógnitas.

### O caso simples: $n = m$

Se $n$ = $m$, não restam nem equações nem incógnitas, e portanto o nosso trabalho acabou.
Na verdade, vamos parar uma etapa antes, com uma equação e uma incógnita,
o que dará o valor desta.
Em seguida, usamos este valor que acabamos de descobrir para determinar a outra incógnita na etapa $(2,2)$.
E assim por diante, vamos determinando sucessivamente os valores de cada uma das incógnitas,
até que as tenhamos todas.

### O caso subdeterminado: $m > n$

Temos mais equações do que incógnitas,
então vamos obter, ao final, $m-n$ incógnitas "livres",
que não estão sujeitas a nenhuma relação.
Podemos proceder de várias formas, mas a mais simples é fixá-las todas em zero
e a partir daí aplicar o procedimento anterior nas outras variávies,
que têm equações associadas.

### O caso superdeterminado: $n > m$

Aqui, temos o contrário: ao final, obtemos $n-m$ equações **sem** incógnitas.
Assim, a menos de ter muita sorte e que estas equações sejam todas equivalentes a $0 = 0$,
este sistema será impossível.
Mais uma vez, há algumas formas de continuar,
dentre as quais, determinar a "solução de mínimos quadrados" associada.

### Exercício

Implemente a substituição para o caso $n = m$.
O seu programa deve testar se a matriz $A$ é triangular superior
e que o par $(A,b)$ corresponde efetivamente ao caso $n = m$.

In [11]:
def subst_solve(A,b):
    # Checar A e b
    #
    #
    #
    
    # Resolver e determinar x
    #
    #
    #
    return x

Verifique resolvendo um sistema aleatório,
e compare com o resultado de `solve`.

In [13]:
A = rand(10,10)
b = rand(10)
elim(A,b)
subst_solve(A, b)

array([-0.3507,  1.3085, -0.4768, -0.1947,  1.4629, -0.1836, -0.0472, -0.4313,  0.4045, -0.2354])

In [14]:
solve(A,b)

array([-0.3507,  1.3085, -0.4768, -0.1947,  1.4629, -0.1836, -0.0472, -0.4313,  0.4045, -0.2354])

Você também pode verificar "na mão" fazendo o produto:

In [15]:
numpy.dot(A,_) - b

array([ -5.5511e-17,   0.0000e+00,  -1.1102e-16,  -4.4409e-16,  -2.7756e-17,   0.0000e+00,
         2.2204e-16,  -1.1102e-16,   2.2204e-16,   0.0000e+00])

O que é melhor, é que este código também resolve sistemas sobredeterminados:

In [16]:
A = rand(10,12)
b = rand(10)
elim(A,b)
subst_solve(A,b)

array([-0.7084, -2.6865,  2.5496,  2.4773,  0.9516, -0.077 , -0.1576, -0.685 , -0.2512, -0.2209,
        0.    ,  0.    ])

In [17]:
dot(A,_) - b

array([  1.1102e-16,   8.3267e-17,  -8.3267e-17,   2.2204e-16,   0.0000e+00,   0.0000e+00,
         0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00])

## Fatoração LU

Um subproduto da eliminação é a fatoração LU: podemos escrever $A = LU$ onde $L$ é triangular inferior (_lower_)
e $U$ é triangular superior (_upper_).

### Exercício

Deduza o algoritmo que permite obter $L$, sabendo que $U$ será a matriz resultante da eliminação.

Deduza também que é possível guardar $L$ e $U$ no mesmo espaço de $A$.

Note que, da mesma forma que é possível resolver $Ux = b'$ com relativa facilidade (substituição),
também é possível resolver $Ly = b$ com a mesma facilidade.
Assim, a fatoração LU permite "quebrar" o problema de resolver $Ax = b$ em duas etapas:
primeiro, resolva $Ly = b$; em seguida, resolva $Ux = y$.

Use, de novo, a matriz fácil para testes:

In [56]:
A = array([[1, 2, 3], [4, 5, 6]])
L,U = lu(A)
print(L)
print(U)
dot(L,U) - A

[[1 0]
 [4 1]]
[[ 1  2  3]
 [ 0 -3 -6]]


array([[0, 0, 0],
       [0, 0, 0]])

E agora, use matrizes aleatórias para verificar que realmente tudo funciona como previsto:

In [57]:
A = rand(4,6)
L,U = lu(A)
print(L)
print(U)
dot(L,U) - A

[[ 1.      0.      0.      0.    ]
 [ 0.407   1.      0.      0.    ]
 [ 0.3955  0.8339  1.      0.    ]
 [ 0.2301  0.1393 -6.8858  1.    ]]
[[ 0.8437  0.4974  0.3309  0.6037  0.9959  0.0448]
 [ 0.      0.7435  0.7299  0.1283  0.4714  0.4718]
 [ 0.      0.     -0.0367 -0.1375  0.0396  0.538 ]
 [ 0.      0.      0.     -0.2258  0.693   3.7601]]


array([[  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,  -1.1102e-16],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   1.1102e-16,   0.0000e+00,  -1.1102e-16]])

In [58]:
A = rand(6,4)
L,U = lu(A)
print(L)
print(U)
dot(L,U) - A

[[  1.0000e+00   0.0000e+00   0.0000e+00   0.0000e+00]
 [  1.9159e+00   1.0000e+00   0.0000e+00   0.0000e+00]
 [  7.1473e-01   1.1364e-01   1.0000e+00   0.0000e+00]
 [  2.4105e-01  -3.4024e-01   1.1895e+00   1.0000e+00]
 [  1.4785e-02  -2.2748e+00   8.4494e+00  -1.4888e+02]
 [  1.2674e+00   7.1379e-01  -1.4653e+00   2.9352e+01]]
[[ 0.4793  0.6043  0.7644  0.236 ]
 [ 0.     -0.3473 -1.131   0.2943]
 [ 0.      0.     -0.2732  0.6088]
 [ 0.      0.      0.      0.0249]]


array([[  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  0.0000e+00,   0.0000e+00,  -1.1102e-16,   0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,  -1.1102e-16],
       [  0.0000e+00,   0.0000e+00,   1.1102e-16,  -3.3307e-16],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00]])

In [59]:
A = rand(6,6)
L,U = lu(A)
print(L)
print(U)
dot(L,U) - A

[[  1.       0.       0.       0.       0.       0.    ]
 [  0.6163   1.       0.       0.       0.       0.    ]
 [  0.1315  12.0204   1.       0.       0.       0.    ]
 [  0.1832  28.853    3.5494   1.       0.       0.    ]
 [  1.2273  16.7912   2.7378   1.1546   1.       0.    ]
 [  1.6979  20.4167   3.0092   1.3024  -4.206    1.    ]]
[[ 0.5608  0.1239  0.2035  0.6827  0.9891  0.5895]
 [ 0.      0.0309  0.1369 -0.265  -0.1336  0.5714]
 [ 0.      0.     -0.8433  3.9629  1.9636 -6.884 ]
 [ 0.      0.      0.     -5.6879 -3.2399  8.4901]
 [ 0.      0.      0.      0.      0.0424 -0.8296]
 [ 0.      0.      0.      0.      0.     -6.1029]]


array([[  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   0.0000e+00,   1.1102e-16],
       [  0.0000e+00,   0.0000e+00,   1.1102e-16,   0.0000e+00,  -1.1102e-16,  -3.3307e-16],
       [  0.0000e+00,   0.0000e+00,  -1.1102e-16,  -5.5511e-16,   1.1102e-16,   7.7716e-16],
       [  0.0000e+00,   0.0000e+00,   1.1102e-16,   1.1102e-16,  -3.3307e-16,   8.8818e-16],
       [  0.0000e+00,   0.0000e+00,  -2.2204e-16,   2.2204e-16,  -2.2204e-16,  -5.5511e-16]])

## Pivôs e permutações

Nem sempre é possível obter a fatoração LU diretamente, pois podemos encontrar um zero na diagonal,
o que torna impossível usar a equação para eliminar.
A saída é permutar as equações para poder continuar.
Isso não muda em nada as variáveis (claro!) mas mudará $b$, pois vamos também permutar seus elementos.
Assim, é preciso memorizar esta operação, o que é feito em geral numa
"matriz de permutações".

### Exercício

Além do critério óbvio que buscamos um coeficiente que seja não-nulo
(na coluna correspondente à variável),
também é comum escolher o coeficiente de maior módulo.
Implemente isso.